#### Лабораторная работа № 3. Разработка нейросетевых функций. Операция Convolution Transpose
#### Новиков М.С. БВТ2003

In [41]:
import torch
import torch.nn.functional as F
import numpy as np

In [42]:
class ConvTranspose:
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, 
                 output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros'):
        "параметры транспонированной свертки"
        self.in_channels = in_channels       
        self.out_channels = out_channels     
        self.kernel_size = kernel_size       
        self.stride = stride                
        self.padding = padding               
        self.output_padding = output_padding 
        self.dilation = dilation             
        self.groups = groups                 
        self.bias = bias                     
        self.padding_mode = padding_mode     
        
        self.weight = torch.nn.Parameter(torch.rand(in_channels, out_channels, *kernel_size))

        if bias:
            self.bias_param = torch.nn.Parameter(torch.rand(out_channels))
        else:
            self.bias_param = None

    def forward(self, input):
        "Nранспонированная свертка на входных данных"
        if self.padding_mode != 'zeros':
            raise ValueError('Поддерживается только режим паддинга "zeros" в ConvTranspose2d')

        filter_for_transpose = torch.flip(self.weight, [2, 3])

        out_tensor = F.conv_transpose2d(
            input, filter_for_transpose, bias=self.bias_param,
            stride=self.stride, padding=self.padding, output_padding=self.output_padding,
            dilation=self.dilation, groups=1  # groups=1, так как в текущей реализации нет поддержки групп
        )

        if self.output_padding > 0:
            pad_func = torch.nn.ConstantPad2d((0, self.output_padding, 0, self.output_padding), 0)
            out_tensor = pad_func(out_tensor)

        out_tensor = out_tensor[:, :, self.padding:out_tensor.shape[2] - self.padding,
                                   self.padding:out_tensor.shape[3] - self.padding]

        rounded_result = torch.round(out_tensor * 1e9) / 1e9

        return rounded_result

In [43]:
def test_conv_transpose():
    "Тестирование класса ConvTranspose на транспонированной свертке"
    conv_transpose = ConvTranspose(
        in_channels=1, out_channels=1, kernel_size=(3, 3),
        stride=2, padding=1, output_padding=1
    )
    x = torch.ones((1, 1, 4, 4)) 
    result = conv_transpose.forward(x)
    print("Тестовый пример:")
    print(f"Input:\n{x}\n")
    print(f"Output:\n{result}")

    filter_for_transpose = torch.flip(conv_transpose.weight, [2, 3])
    expected_result = F.conv_transpose2d(
        x, filter_for_transpose, bias=conv_transpose.bias_param,
        stride=conv_transpose.stride, padding=conv_transpose.padding,
        output_padding=conv_transpose.output_padding, dilation=conv_transpose.dilation, groups=1
    )
    print(f"\nОжидаемый результат:\n{expected_result}")


In [44]:
test_conv_transpose()

Тестовый пример:
Input:
tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]])

Output:
tensor([[[[1.9207, 1.8786, 1.9207, 1.8786, 1.9207, 1.8786, 1.6648],
          [1.1782, 1.4292, 1.1782, 1.4292, 1.1782, 1.4292, 0.7736],
          [1.9207, 1.8786, 1.9207, 1.8786, 1.9207, 1.8786, 1.6648],
          [1.1782, 1.4292, 1.1782, 1.4292, 1.1782, 1.4292, 0.7736],
          [1.9207, 1.8786, 1.9207, 1.8786, 1.9207, 1.8786, 1.6648],
          [1.1782, 1.4292, 1.1782, 1.4292, 1.1782, 1.4292, 0.7736],
          [1.4218, 1.2507, 1.4218, 1.2507, 1.4218, 1.2507, 1.2058]]]],
       grad_fn=<DivBackward0>)

Ожидаемый результат:
tensor([[[[1.4292, 1.1782, 1.4292, 1.1782, 1.4292, 1.1782, 1.4292, 0.7736],
          [1.8786, 1.9207, 1.8786, 1.9207, 1.8786, 1.9207, 1.8786, 1.6648],
          [1.4292, 1.1782, 1.4292, 1.1782, 1.4292, 1.1782, 1.4292, 0.7736],
          [1.8786, 1.9207, 1.8786, 1.9207, 1.8786, 1.9207, 1.8786, 1.6648],
          [1.429

#### Дополнительное задание :
#### Необходимо реализовать алгоритм работы транспонированной свертки через алгоритм двумерной свертки, реализованный в первой лабораторной

In [1]:
import torch
import numpy as np
from typing import Union

In [13]:
class Conv2DTranspose:
    def __init__(self, in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple] = 1) -> None:
        "Конструктор класса Conv2DTranspose"
        self.in_channels = in_channels     
        self.out_channels = out_channels   
        self.kernel_size = kernel_size     
        self.stride = stride               

        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)

        self.weights = np.random.randn(in_channels, out_channels, *kernel_size) 
        self.bias = np.zeros((out_channels,))

    def forward(self, input_tensor: np.ndarray) -> np.ndarray:
        "Метод для выполнения операции транспонированной свертки"
        batch_size, in_channels, input_height, input_width = input_tensor.shape
        kernel_size = self.kernel_size

        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)

        if isinstance(self.stride, int):
            stride_h = stride_w = self.stride
        elif isinstance(self.stride, tuple) and len(self.stride) == 2:
            stride_h, stride_w = self.stride
        else:
            raise ValueError("Недопустимый формат шага. Должно быть целое число или кортеж из двух целых чисел")

        output_height = (input_height - 1) * stride_h + kernel_size[0]
        output_width = (input_width - 1) * stride_w + kernel_size[1]

        result = np.zeros((batch_size, self.out_channels, output_height, output_width))

        for i in range(input_height):
            for j in range(input_width):
                window = input_tensor[:, :, i, j]
                result[:, :, i * stride_h:i * stride_h + kernel_size[0], j * stride_w:j * stride_w + kernel_size[1]] += \
                    np.tensordot(window, self.weights, axes=(1, 0))

        return result

In [14]:
def run_transpose_test(in_channels: int, out_channels: int, kernel_size: Union[int, tuple], stride: Union[int, tuple],
                        padding: Union[int, tuple], dilation: int, groups: int, bias: bool, padding_mode: str) -> None:
    "Функция для выполнения теста операции транспонированной свертки"
    conv_transpose_layer = Conv2DTranspose(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride)

    input_tensor = np.random.randn(1, in_channels, 10, 10)

    result_conv2d_transpose = conv_transpose_layer.forward(input_tensor)

    input_tensor_torch = torch.tensor(input_tensor)
    if isinstance(conv_transpose_layer.stride, int):  
        stride_h = stride_w = conv_transpose_layer.stride
    elif isinstance(conv_transpose_layer.stride, tuple) and len(conv_transpose_layer.stride) == 2:
        stride_h, stride_w = conv_transpose_layer.stride
    else:
        raise ValueError("Недопустимый формат шага. Должно быть целое число или кортеж из двух целых чисел")

    output_tensor_torch = torch.nn.functional.conv_transpose2d(input_tensor_torch,
                                                               torch.tensor(conv_transpose_layer.weights),
                                                               bias=torch.tensor(conv_transpose_layer.bias),
                                                               stride=(stride_h, stride_w),
                                                               padding=padding,
                                                               output_padding=stride_h - 1,
                                                               dilation=dilation,
                                                               groups=groups).numpy()

    min_height = min(result_conv2d_transpose.shape[2], output_tensor_torch.shape[2])
    min_width = min(result_conv2d_transpose.shape[3], output_tensor_torch.shape[3])
    result_conv2d_transpose = result_conv2d_transpose[:, :, :min_height, :min_width]
    output_tensor_torch = output_tensor_torch[:, :, :min_height, :min_width]

    are_results_equal = np.allclose(result_conv2d_transpose, output_tensor_torch)

    print(f"Тест с параметрами:")
    print(f"  - in_channels: {in_channels}")
    print(f"  - out_channels: {out_channels}")
    print(f"  - kernel_size: {kernel_size}")
    print(f"  - stride: {stride}")
    print(f"  - padding: {padding}")
    print(f"  - dilation: {dilation}")
    print(f"  - groups: {groups}")
    print(f"  - bias: {bias}")
    print(f"  - padding_mode: {padding_mode}")
    print(f"\nРезультат транспонированной свертки совпадает: {are_results_equal}")
    print(f"\n{'-'*75}\n")

In [19]:
def test_transpose_params():
    "Функция для тестирования различных параметров операции свертки"
    run_transpose_test(in_channels=3, out_channels=2, kernel_size=3, stride=(2, 2), padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

    run_transpose_test(in_channels=4, out_channels=2, kernel_size=(2, 3), stride=(2, 3), padding=(1, 1), dilation=1, groups=1, bias=True, padding_mode='replicate')


In [20]:
test_transpose_params()

Тест с параметрами:
  - in_channels: 3
  - out_channels: 2
  - kernel_size: 3
  - stride: (2, 2)
  - padding: 0
  - dilation: 1
  - groups: 1
  - bias: True
  - padding_mode: zeros

Результат транспонированной свертки совпадает: True

---------------------------------------------------------------------------

Тест с параметрами:
  - in_channels: 4
  - out_channels: 2
  - kernel_size: (2, 3)
  - stride: (2, 3)
  - padding: (1, 1)
  - dilation: 1
  - groups: 1
  - bias: True
  - padding_mode: replicate

Результат транспонированной свертки совпадает: False

---------------------------------------------------------------------------

